In [6]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
# install other resources for doing crawling
!apt install chromium-chromedriver

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By 

In [8]:
DRIVER = None
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

In [4]:
def close_driver():
    global DRIVER
    if not DRIVER is None:
        DRIVER.quit()

    DRIVER = None

In [9]:
initialize_driver()
DRIVER.get('https://vnexpress.net/')
all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'item-news')
news_element = all_news_elements[3]

print(news_element.get_attribute('outerHTML')) 

Initiating driver...
Finished!
<article data-swap="4" class="item-news item-news-common"><h3 class="title-news"><a href="https://vnexpress.net/doanh-nghiep-gong-lai-di-vay-4552213.html" title="Doanh nghiệp 'gồng' lãi đi vay" data-medium="Item-4" data-thumb="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-4&amp;vn_term=Desktop&amp;vn_thumb=1" data-itm-added="1">Doanh nghiệp 'gồng' lãi đi vay</a></h3><div class="thumb-art"><a href="https://vnexpress.net/doanh-nghiep-gong-lai-di-vay-4552213.html" class="thumb thumb-5x3" title="Doanh nghiệp 'gồng' lãi đi vay" data-medium="Item-4" data-thumb="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-4&amp;vn_term=Desktop&amp;vn_thumb=1" data-itm-added="1"><picture><!--[if IE 9]><video style="display: none;"><![endif]--><source srcset="https://i1-kinhdoanh.vnecdn.net/2022/12/25/vpbank-thanhtung11-1671871449-7787-9986-1671908397.jpg?w=220&amp;h=132&amp;q=100&amp;dpr=1&amp;fit=crop&amp;

In [10]:
description_element = news_element.find_element(By.CLASS_NAME, 'description') 

In [11]:
a_element = description_element.find_element(By.TAG_NAME, 'a')
article_link = a_element.get_attribute('href')
print(article_link)

https://vnexpress.net/doanh-nghiep-gong-lai-di-vay-4552213.html


In [12]:
title = a_element.get_attribute('title')
print(title)

Doanh nghiệp 'gồng' lãi đi vay>


In [13]:
description = a_element.text
print(description)

Vay ngân hàng đã "dễ thở" hơn nhưng lãi suất cao cộng sức mua yếu đang tạo áp lực kép khiến doanh nghiệp đứng trước bài toán giảm quy mô.


In [14]:
def get_link_title_description(news_element):
    '''
    Return link, title and description of an article from a web element
    '''

    description_element = news_element.find_element(By.CLASS_NAME, 'description') 

    a_element = description_element.find_element(By.TAG_NAME, 'a')
    # article link
    article_link = a_element.get_attribute('href')

    # title
    title = a_element.get_attribute('title')

    # description
    desc = a_element.text

    return article_link,title,desc

In [15]:
a = get_link_title_description(news_element)
a

('https://vnexpress.net/doanh-nghiep-gong-lai-di-vay-4552213.html',
 "Doanh nghiệp 'gồng' lãi đi vay>",
 'Vay ngân hàng đã "dễ thở" hơn nhưng lãi suất cao cộng sức mua yếu đang tạo áp lực kép khiến doanh nghiệp đứng trước bài toán giảm quy mô.')

In [16]:
thumbart_class = news_element.find_element(By.CLASS_NAME, 'thumb-art') 
thumbnail_link = thumbart_class.find_element(By.TAG_NAME, 'img').get_attribute('src')
print(thumbnail_link)

https://i1-kinhdoanh.vnecdn.net/2022/12/25/vpbank-thanhtung11-1671871449-7787-9986-1671908397.jpg?w=120&h=72&q=100&dpr=2&fit=crop&s=O5yZsmBhJMGbF8OfgYHNMg


In [17]:
def get_thumbnail_link(news_element):
    '''
    Return thumbnail link (if possible) given the web element
    '''

    thumbnail_link = ''
    try:
       thumbart_class = news_element.find_element(By.CLASS_NAME, 'thumb-art') 
       thumbnail_link = thumbart_class.find_element(By.TAG_NAME, 'img').get_attribute('src')
    except Exception as e:
        # there's an error
        print('Cannot find thumbnail_link')
    

    return thumbnail_link

In [18]:
print(get_thumbnail_link(news_element))

https://i1-kinhdoanh.vnecdn.net/2022/12/25/vpbank-thanhtung11-1671871449-7787-9986-1671908397.jpg?w=120&h=72&q=100&dpr=2&fit=crop&s=O5yZsmBhJMGbF8OfgYHNMg


In [19]:
def scrape_vnexpress(DRIVER):
    '''
    Return a list of items, each item is another list containing 4 items: 
    - Article link
    - Title
    - Description
    - Thumbnail link (if possible)
    '''

    results = []
    all_news_elements = DRIVER.find_elements(By.CLASS_NAME,'item-news')
    print(f'There are {len(all_news_elements)} elements found')
    n_skip=0
    n_success=0
    for news_element in all_news_elements:
        print('Getting new elements...')
        try:

            link,title,desc = get_link_title_description(news_element)
            thumbnail_link = get_thumbnail_link(news_element)
            results.append([link,title,desc,thumbnail_link])
            n_success+=1
        except Exception as e:
            print('Cannot get this article. Skip')
            # print(news_element.get_attribute('outerHTML'))
            n_skip+=1
            
    
    print(f'Number of scraped articles: {n_success}')
    print(f'Number of skipped articles: {n_skip}')
    return results

In [20]:
results = scrape_vnexpress(DRIVER)

There are 219 elements found
Getting new elements...
Getting new elements...
Getting new elements...
Cannot get this article. Skip
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Cannot get this article. Skip
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Getting new elements...
Cannot find thumbnail_link
Getting new elements...
Cannot get this article. Skip
Getting new elements...
Cannot get this article. Skip
Getting new elements...
Cannot get this article. Skip
Getting new elements...
Getting new elements...
Ca

In [22]:
import pandas as pd
df = pd.DataFrame(results,columns=['link','title','description','thumbnail_link'])
df

,link,title,description,thumbnail_link
0,https://vnexpress.net/hlv-park-du-doan-doi-hin...,HLV Park dự đoán đội hình của Malaysia,"HÀ NỘITheo HLV Park Hang-seo, Malaysia đã dưỡn...",https://i1-thethao.vnecdn.net/2022/12/26/32107...
1,https://vnexpress.net/qua-tet-bieu-sep-4551481...,Quà Tết biếu sếp,"Một năm, tôi quyết định không biếu Tết sếp nào...",https://i1-vnexpress.vnecdn.net/2022/09/18/Qua...
2,https://vnexpress.net/doanh-nghiep-gong-lai-di...,Doanh nghiệp 'gồng' lãi đi vay>,"Vay ngân hàng đã ""dễ thở"" hơn nhưng lãi suất c...",https://i1-kinhdoanh.vnecdn.net/2022/12/25/vpb...
3,https://vnexpress.net/ong-dang-ha-viet-mat-co-...,Ông Đặng Hà Việt: ‘Mặt cỏ sân Mỹ Đình không xa...,Theo Tổng cục trưởng Tổng cục TDTT Đặng Hà Việ...,https://i1-thethao.vnecdn.net/2022/12/26/1bb4a...
4,https://vnexpress.net/ong-medvedev-phuong-tay-...,Ông Medvedev: Phương Tây muốn phát động chiến ...,Ông Medvedev nói phương Tây muốn phát động chi...,https://i1-vnexpress.vnecdn.net/2022/12/26/7yb...
...,...,...,...,...
78,https://ngoisao.vnexpress.net/le-quyen-ban-tra...,Lệ Quyên: 'Bạn trai hy sinh nhiều khi yêu tôi',,https://i1-ngoisao.vnecdn.net/2022/12/26/img-0...
79,https://ngoisao.vnexpress.net/diem-my-9x-duoc-...,Diễm My 9X được bạn trai Việt kiều cầu hôn,,https://i1-ngoisao.vnecdn.net/2022/12/26/diem-...
80,https://e.vnexpress.net/news/trend/unemployed-...,Unemployed workers swim with hungry loan sharks,Vietnamese workers who lose their jobs often f...,https://i1-english.vnecdn.net/2022/12/26/1-167...
81,https://e.vnexpress.net/news/football/japanese...,Japanese football club signs Vietnamese striker,,https://i1-english.vnecdn.net/2022/12/26/attkp...
